# Expansion vizualisation algorithm

The purpose of this notebook is first of all to align a stack (movie composed of several images) taken by a microscope in order to correct the movements of the microscope between two images. Then, once the images are aligned, each image will be binarized (transformed into a matrix composed of 0 and 1) to distinguish the microcolonies in the background.  Finally, the binarized images will be added together in order to have a global vision of the evolution of each microcolony and to be able to extract information such as the growth rate.

## Packages

In [ ]:
import skimage
from skimage import io
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb

## Additional package

The following package is not available directly with the Anaconda distribution and must be installed separately. To install it, simply type the following command in an empty cell:

<em>pip install pystackreg</em>


More information about this package is available here: https://pypi.org/project/pystackreg/

In [ ]:
from pystackreg import StackReg

## Alignement and binarization of a stack

In [ ]:
# directory of the stack (movie) 
rootdir = '/path/to/images.tif'
img0 = io.imread(rootdir)
# alignement to the previous image
sr = StackReg(StackReg.TRANSLATION)
out_previous = sr.register_transform_stack(img0, reference='previous')
# binarization
binary_images = [];
for image in out_previous:
    try:
        # apply otsu algoritme
        thresh = skimage.filters.threshold_otsu(image)
        binary = image > thresh
        # remove artifacts connected to image border
        cleared = clear_border(binary)
    except:
        print("An exception occurred") 
    binary_images.append(cleared)
    # sum all the images of the current position to creat a 2D histogram
    sum_binary = sum(binary_images)